<a href="https://colab.research.google.com/github/KiralyOr/PBP/blob/bayesian_network/BPtoBN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

In [ ]:
from collections import defaultdict
import itertools
import graphviz
import pandas as pd

#Monty Hall pgmpy


In [ ]:
dot = graphviz.Digraph(comment='#Monty Hall pgmpy')

dot.node('G', 'Guess')
dot.node('H', 'Hide')
dot.node('O', 'Open')

dot.edges(['GO', 'HO'])
dot

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Defining the network structure
model = BayesianNetwork([("G", "O"), ("H", "O")])

# Defining the CPDs:
cpd_g = TabularCPD("G", 3, [[0.33], [0.33], [0.33]])
cpd_h = TabularCPD("H", 3, [[0.33], [0.33], [0.33]])
cpd_o = TabularCPD(
    "O",
    3,
    [
        [0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
        [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
        [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0],
    ],
    evidence=["G", "H"],
    evidence_card=[3, 3],
)

# Associating the CPDs with the network structure.
model.add_cpds(cpd_g, cpd_h, cpd_o)

# Some other methods
model.get_cpds()

In [ ]:
# check_model check for the model structure and the associated CPD and returns True if everything is correct otherwise throws an exception
model.check_model()

In [ ]:
# Infering the posterior probability
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior_h = infer.query(["H"], evidence={"G": 0, "O": 2})
print(posterior_h)

#BThreds implementation


###Data Structure definition

In [ ]:
INITIALS={'bt1':'0', 'bt2':'0', 'bt3':'0'}

BTHREADS= ['bt1','bt2','bt3']


EVENTS = ['default', 'g1', 'g2', 'g3', 'h1', 'h2', 'h3', 'o1', 'o2', 'o3']

STATES = defaultdict(list,
            {'bt1':['0','1','2','3'],
             'bt2':['0','1','2','3'],
             'bt3':['0','1','2','3']})
                         
                     

BLOCKED =defaultdict(list,
            {'bt3_state_1': ['o1'],
             'bt3_state_2': ['o2'],
             'bt3_state_3': ['o3'],
             'bt2_state_1': ['o1'],
             'bt2_state_2': ['o2'],
             'bt2_state_3': ['o3']})

REQUESTED=defaultdict(list,
            {'bt1_state_0': ['h1', 'h2', 'h3'],
             'bt1_state_1': ['g1', 'g2', 'g3'],
             'bt1_state_2': ['o1', 'o2', 'o3']})

TIME=[0, 1, 2]
TRANSITIONS={('bt1_state_0', 'bt1_state_1', 'h1'),
 ('bt1_state_0', 'bt1_state_1', 'h2'),
 ('bt1_state_0', 'bt1_state_1', 'h3'),
 ('bt1_state_1', 'bt1_state_2', 'g1'),
 ('bt1_state_1', 'bt1_state_2', 'g2'),
 ('bt1_state_1', 'bt1_state_2', 'g3'),
 ('bt1_state_2', 'bt1_state_3', 'o1'),
 ('bt1_state_2', 'bt1_state_3', 'o2'),
 ('bt1_state_2', 'bt1_state_3', 'o3'),
 ('bt2_state_0', 'bt2_state_1', 'h1'),
 ('bt2_state_0', 'bt2_state_2', 'h2'),
 ('bt2_state_0', 'bt2_state_3', 'h3'),
 ('bt3_state_0', 'bt3_state_1', 'g1'),
 ('bt3_state_0', 'bt3_state_2', 'g2'),
 ('bt3_state_0', 'bt3_state_3', 'g3')}

# Translation to Bayesian Network

In [ ]:
p = graphviz.Graph(name='Bayesian Network')

for i in range(1,4):
  p.node(f'e_{i}', f'event({i})')

for bt in BTHREADS:
  for i in range(4):
    p.node(f'{bt}_{i}', f'state({bt},{i})')

for i in range(1,4):
  for bt in BTHREADS:
    p.edge(f'e_{i}', f'{bt}_{i}')

for bt in BTHREADS:
  for i in range(3):
    p.edge(f'{bt}_{i}',f'e_{i+1}')
    p.edge(f'{bt}_{i}',f'{bt}_{i+1}')
p

##BP util functions

In [ ]:
def requested_not_blocked(row) -> set:
  # get set of all requested not blocked  from given state
  states = [row[bt] for bt in BTHREADS]
  requested = list()
  blocked = list()
  for bt,s in enumerate(states):
    requested.extend(REQUESTED[f'bt{bt+1}_state_{s}'])
    blocked.extend(BLOCKED[f'bt{bt+1}_state_{s}'])
  return set(requested) - set(blocked)

In [ ]:
def event_prob(event,rnb_set):
  # calculate event probability 
  # using uniform distribution for all 
  # requested not blocked events
  if event in rnb_set:
    return 1/len(rnb_set)
  elif len(rnb_set) == 0 and event == 'default':
    return 1
  else:  
    return 0

In [ ]:
def check_no_tarnsiton(bt_name, source, event):
  # checks that there aren't any transition from given state with given event
  for target in STATES[bt_name]:
    if (f"{bt_name}_state_{source}", f"{bt_name}_state_{target}" , event) in TRANSITIONS:
      return False
  return True
check_no_tarnsiton('bt1', '0', 'h1')

In [ ]:
def is_valid_transition(bt_name, source, target, event):
  # check if transition is valid using TRANSITIONS data structure
  # if the transition not exists checks if self loop should be assigend
  if (f"{bt_name}_state_{source}", f"{bt_name}_state_{target}" , event) in TRANSITIONS:
    return 1
  elif source == target and check_no_tarnsiton(bt_name, source, event):
    return 1
  else:   
    return 0

##Create CPDs

In [ ]:
def pivot_cpd(df):
  pivot_df = pd.pivot_table(df, values='prob', index=['event'],
                    columns=BTHREADS).reset_index()                 
  return pivot_df

def get_cpd_event_table():
  states = [STATES[bt] for bt in BTHREADS]
  states.append(EVENTS)
  event_table = [p for p in itertools.product(*states)]
  columns = BTHREADS +['event']
  df = pd.DataFrame(event_table, columns=columns)
  df['prob'] = df.apply(lambda x: event_prob(x['event'], requested_not_blocked((x))), axis=1)
  df.sort_values(by=columns)
  return pivot_cpd(df)

In [ ]:
def get_cpd_table_internal_state(bt_name):
  states = [STATES[bt_name],list(EVENTS), STATES[bt_name]]
  event_table = [p for p in itertools.product(*states)]
  columns = [bt_name , 'event', f'next_{bt_name}']
  df = pd.DataFrame(event_table, columns=columns)
  df['prob'] = df.apply(lambda x: is_valid_transition(bt_name, x[bt_name], x[f'next_{bt_name}'], x['event']) ,axis=1) 
  pivot_df = pd.pivot_table(df, values='prob', index=[f'next_{bt_name}'],
                    columns=[bt_name, 'event']).reset_index().fillna(0)
  return pivot_df

In [ ]:
def get_bt_initial_cpd(bt_name):
  df = pd.DataFrame(STATES[bt_name], columns=[bt_name])
  df['prob'] = df.apply(lambda x: 1 if INITIALS[bt_name] == x[bt_name] else 0 ,axis=1) 
  return df
get_bt_initial_cpd('bt1')  

### Convert Dataframe to list of lists

In [ ]:
df_bt1_initial = get_bt_initial_cpd('bt1')
df_bt1_initial

In [ ]:
df_bt2_initial = get_bt_initial_cpd('bt2')
df_bt2_initial

In [ ]:
df_bt3_initial = get_bt_initial_cpd('bt3')
df_bt3_initial

In [ ]:
df_event = get_cpd_event_table()
df_event

In [ ]:
df_bt1 = get_cpd_table_internal_state('bt1') 
df_bt1

In [ ]:
df_bt2 = get_cpd_table_internal_state('bt2') 
df_bt2

In [ ]:
df_bt3 = get_cpd_table_internal_state('bt3') 
df_bt3

In [ ]:
cpd_df_bt1_initial = [l[1:] for l in df_bt1_initial.values.tolist()]
cpd_df_bt2_initial = [l[1:] for l in df_bt2_initial.values.tolist()]
cpd_df_bt3_initial = [l[1:] for l in df_bt3_initial.values.tolist()]

cpd_df_event = [l[1:] for l in df_event.values.tolist()]

cpd_df_bt1 = [l[1:] for l in df_bt1.values.tolist()]
cpd_df_bt2 = [l[1:] for l in df_bt2.values.tolist()]
cpd_df_bt3 = [l[1:] for l in df_bt3.values.tolist()]



### Build Bayesian Network

In [ ]:
# Defining the network structure
BN = []
for i in range(1,4):
  for bt in BTHREADS:
    BN.append((f'event_{i}', f'state_{bt}_{i}'))

for bt in BTHREADS:
  for i in range(3):
    BN.append((f'state_{bt}_{i}',f'event_{i+1}'))
    BN.append((f'state_{bt}_{i}',f'state_{bt}_{i+1}'))
model = BayesianNetwork(BN)
BN

In [ ]:
# Defining the CPDs:
cpd_state_bt1_0 = TabularCPD("state_bt1_0", 4, cpd_df_bt1_initial)
cpd_state_bt2_0 = TabularCPD("state_bt2_0", 4, cpd_df_bt2_initial)
cpd_state_bt3_0 = TabularCPD("state_bt3_0", 4, cpd_df_bt3_initial)
cpd_event_1 = TabularCPD("event_1", 10, cpd_df_event, evidence=["state_bt1_0", "state_bt2_0", "state_bt3_0"], evidence_card=[4, 4, 4])

In [ ]:
cpd_state_bt1_1 = TabularCPD("state_bt1_1", 4, cpd_df_bt1, evidence=["state_bt1_0", "event_1"], evidence_card=[4,10])
cpd_state_bt2_1 = TabularCPD("state_bt2_1", 4, cpd_df_bt2, evidence=["state_bt2_0", "event_1"], evidence_card=[4,10])
cpd_state_bt3_1 = TabularCPD("state_bt3_1", 4, cpd_df_bt3, evidence=["state_bt3_0", "event_1"], evidence_card=[4,10])
cpd_event_2 = TabularCPD("event_2", 10, cpd_df_event, evidence=["state_bt1_1", "state_bt2_1", "state_bt3_1"], evidence_card=[4, 4, 4])

In [ ]:
cpd_state_bt1_2 = TabularCPD("state_bt1_2", 4, cpd_df_bt1, evidence=["state_bt1_1", "event_2"], evidence_card=[4, 10])
cpd_state_bt2_2 = TabularCPD("state_bt2_2", 4, cpd_df_bt2, evidence=["state_bt2_1", "event_2"], evidence_card=[4, 10])
cpd_state_bt3_2 = TabularCPD("state_bt3_2", 4, cpd_df_bt3, evidence=["state_bt3_1", "event_2"], evidence_card=[4, 10])
cpd_event_3 = TabularCPD("event_3", 10, cpd_df_event, evidence=["state_bt1_2", "state_bt2_2", "state_bt3_2"], evidence_card=[4, 4, 4])

In [ ]:
cpd_state_bt1_3 = TabularCPD("state_bt1_3", 4, cpd_df_bt1, evidence=["state_bt1_2", "event_3"], evidence_card=[4, 10])
cpd_state_bt2_3 = TabularCPD("state_bt2_3", 4, cpd_df_bt2, evidence=["state_bt2_2", "event_3"], evidence_card=[4, 10])
cpd_state_bt3_3 = TabularCPD("state_bt3_3", 4, cpd_df_bt3, evidence=["state_bt3_2", "event_3"], evidence_card=[4, 10])

In [ ]:
# Associating the CPDs with the network structure.
model.add_cpds(
    cpd_state_bt1_0,
    cpd_state_bt2_0,
    cpd_state_bt3_0,
    cpd_event_1,
    cpd_state_bt1_1,
    cpd_state_bt2_1,
    cpd_state_bt3_1,
    cpd_event_2,
    cpd_state_bt1_2,
    cpd_state_bt2_2,
    cpd_state_bt3_2,
    cpd_event_3,
    cpd_state_bt1_3,
    cpd_state_bt2_3,
    cpd_state_bt3_3,
)

In [ ]:
# check_model check for the model structure and the associated CPD and returns True if everything is correct otherwise throws an exception
model.check_model()

## Model evaluation


In [ ]:
# Infering the posterior probability
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
# EVENTS = ['default', 'g1', 'g2', 'g3', 'h1', 'h2', 'h3', 'o1', 'o2', 'o3']
posterior_h = infer.query(["event_1"], evidence={"event_2": 2, "event_3": 9})
print(posterior_h)

In [ ]:
posterior_h = infer.query(["event_3"], evidence={"event_2": 1, "event_1": 4})
print(posterior_h)